# Wiener filters

This notebook implements the most basic Wiener filter, which is intended to filter observed data $d_i^\text{obs}$ for $i=0, ..., N-1$ using the $n$-point discrete convolution

\begin{equation}
d_i = \sum_{j=0}^{n-1} d_{i-j}^\text{obs} p_j\,.
\end{equation}

Wiener filters are designed to minimise the difference between $d_i$ and some desired signal $x_i$. Typically, $x_i$ an unknown noise-free version of $d_i^\text{obs}$.

The task of finding the $n$ filter coefficients $p_j$ can be formulated as an inverse problem. Assuming that the observational errors are Gaussian with covariance matrix $\mathbf{C}$, the maximum-likelihood set of coefficients minimises the least-squares misfit functional

\begin{equation}
\chi(\mathbf{p}) = \frac{1}{2} \sum_{i,j=0}^{N-1} (d_i - x_i) C_{ij}^{-1} (d_j - x_j)\,.
\end{equation}

Inserting the forward modelling equation and forcing the derivative of $\chi$ to zero, yields the normal equations

\begin{equation}
\sum_{i,j=0}^{N-1} x_i C_{ij}^{-1} d_{j-q}^\text{obs} = \sum_{k=0}^{n-1} \left( \sum_{i,j=0}^{N-1} d_{i-k}^\text{obs} C_{ij}^{-1} d_{j-q}^\text{obs} \right)\,p_k\,.
\end{equation}

This can be written in vector-matrix form,

\begin{equation}
b_q = \sum_{k=0}^{n-1} A_{qk} p_k\,,
\end{equation}

with the cross-correlation vector between observed and ideal data,

\begin{equation}
b_q = \sum_{i,j=0}^{N-1} x_i C_{ij}^{-1} d_{j-q}^\text{obs}\,,
\end{equation}

and the auto-correlation matrix between the observed data,

\begin{equation}
A_{qk}=\sum_{i,j=0}^{N-1} d_{i-k}^\text{obs} C_{ij}^{-1} d_{j-q}^\text{obs}\,.
\end{equation}

The crux of the problem is of course that the ideal data are unknown. Yet, we may obtain an estimate of $b_q$ under the assumption that the noise time series $n_i$ is additive to the ideal data in the sense of

\begin{equation}
d_i^\text{obs} = x_i + n_i\,.
\end{equation}

Furthermore, when noise and data are uncorrelated, we may change the expression for $b_q$ to

\begin{equation}
b_q = \sum_{i,j=0}^{N-1} x_i C_{ij}^{-1} x_{j-q}\,.
\end{equation}

It follows that the filter may be constructed just from the cross-correlation properties of the ideal data, without actually knowing them explicitly.

The data needed for this notebook can be downloaded with this Polybox link: https://polybox.ethz.ch/index.php/s/a3EDhinFdATu4qt .

# 0. Packages and setup

Import the necessary Python packages and add a few lines to embellish figures.

In [ ]:
import obspy
import numpy as np
import time as time
import matplotlib.pyplot as plt
from obspy.signal.filter import bandpass

plt.rcParams["font.family"] = "Times"
plt.rcParams.update({'font.size': 50})
plt.rcParams['xtick.major.pad']='12'
plt.rcParams['ytick.major.pad']='12'

# 1. General input

Basic input, including the file to be read and the average spacing between channels.

In [ ]:
# GRIMSVÖTN DATA ========================================

# Input file.
input_file='/Users/andreas/Desktop/PEF/2021-05-26T00_02_14.998000Z.npy'
# Scale for plotting.
scale=2
# Receiver spacing.
dx=8.0
# Time increment.
dt=0.01

# 2. Data reading and plotting

## 2.1. Read and plot complete data

We start by reading in all the data and plotting them for an initial inspection.

In [ ]:
cct=np.load(input_file)

nt=cct.shape[1]
nx=cct.shape[0]-1

t=np.linspace(0.0,nt*dt,nt)

Plotting all data might take a long time. So, one may want to deactivate this part of the notebook.

## 2.2. Filtering and downsampling

For some applications, we may want to reduce the data volume through the application of a bandpass and by down-sampling. This is not strictly needed, however.

In [ ]:
# Minimum and maximum frequencies [Hz].
freqmin=0.1
freqmax=49.0
# Downsampling factor.
downsample=1

In [ ]:
# Frequency-domain filtering.
cct_filt=np.zeros(np.shape(cct))
for i in range(cct.shape[0]-1): cct_filt[i,:]=bandpass(cct[i,:],freqmin=freqmin,freqmax=freqmax,df=1.0/dt,corners=4,zerophase=True)

In [ ]:
# Downsampling.
cct_filt_down=cct_filt[:,0:nt:downsample]

nx=cct_filt_down.shape[0]-1
nt=cct_filt_down.shape[1]
dt=downsample*dt

t=np.linspace(0.0,nt*dt,nt)

We then plot the pre-processed data. Also this may take a long time.

## 2.3. Pick a specific trace

As data, we first pick one of the available traces. We will use this trace to estimate the noise covariance matrix $\mathbf{C}$, as well as the vector $\mathbf{b}$ that we need for the Wiener filter coefficients.

In [ ]:
# Index of the trace.
i_trace=360
# Pick trace.
d=cct_filt_down[i_trace,:]

# Plot individual trace.
plt.figure(figsize=(30,10))
plt.plot(d,'k')
plt.grid()
plt.xlabel('sample index')
plt.show()

# 3. Noise covariance matrix

We first need an estimate of the data covariance matrix $\mathbf{C}$. For this, we first need to determine the size of the training dataset $N$. We then take a window of this length and slide it across the noise part of the signal, prior to the first wave arrivals. From this, we get average cross-correlations between pairs of samples.

In [ ]:
# Number of data points in the dataset. This is needed to determine the size of the data covariance matrix.
Nd=400

In [ ]:
# Correlation vector.
corr=np.zeros(Nd)

# Minimum and maximum indices from the noise time series.
imin=100
imax=3900

# Compute mean and remove from data.
mean=np.mean(d[imin:imax])
d-=mean

# Compute correlation vector.
for j in np.arange(imin,imax):
    for i in range(Nd):
        corr[i]+=d[j]*d[j+i]

corr/=np.float(imax-imin)

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(corr,'k')
plt.grid()
plt.title('average noise cross-correlation',pad=30)
plt.xlabel('sample index')
plt.xlim([0,Nd-1])
plt.show()

We finally build $\mathbf{C}$ from the average cross-correlation by putting shifted copies of it into the rows of the matrix.

In [ ]:
C=np.zeros([Nd,Nd])
for i in range(Nd): C[i,:]=np.roll(corr,i)
Cinv=np.linalg.inv(C)

# 4. Filtering

## 4.1. Computing filter coefficients

We start by picking out a part of the time series that we are interested in. Based on this, we build the vector $\mathbf{b}$ and the matrix $\mathbf{A}$.

In [ ]:
# Starting index of the time series of interest.
i0=3900

# Plot the section of the time series of interest.
plt.figure(figsize=(30,10))
plt.plot(np.arange(i0-50,i0+Nd+50),d[i0-50:i0+Nd+50],'--k')
plt.plot(np.arange(i0,i0+Nd),d[i0:i0+Nd],'k',LineWidth=4)
plt.grid()
plt.xlim([i0-50,i0+Nd+50])
plt.title('data',pad=20)
plt.xlabel('sample index')
plt.show()

In the next step, we assemble $\mathbf{b}$ and $\mathbf{A}$. To make this more efficient, we use the fact that the matrix $X_{i,q}=\sum_{j=0}^{N} C_{ij}^{-1} d_{j-q}^\text{obs}$ appears in both $\mathbf{b}$ and $\mathbf{A}$.

In [ ]:
# Number of Wiener filter coefficients.
n=10

Define a function that builds the matrix $\mathbf{A}$ for a specific input datum.

In [ ]:
def matrix(d,plot=False):

    # Build the auxiliary matrix X.
    X=np.zeros([Nd,n])
    for q in range(n): X[:,q]=np.dot(Cinv,d[(i0-q):(i0-q+Nd)])

    # Build matrix A.
    A=np.zeros([n,n])
    for k in range(n): A[:,k]=np.dot(d[(i0-k):(i0-k+Nd)],X)

    # Plot matrix.
    if plot:
        plt.figure(figsize=(15,15))
        plt.pcolor(A)
        plt.title('auto-correlation matrix A',pad=30)
        plt.show()

    # Return.
    return A

Build the vector $\mathbf{b}$ using a rough estimate of the perfect data. We obtain this estimate by picking out a part of the time series that we think is dominated by actual data.

In [ ]:
# Cut out a part of the trace.
d0=d.copy()
d0[:4129]=0.0
d0[4140:]=0.0

# Smooth a bit.
for s in range(3): d0[1:nt-1]=(2.0*d0[1:nt-1]+d0[0:nt-2]+d0[2:nt])/4.0

# Plot the result.
plt.figure(figsize=(30,10))
plt.plot(np.arange(i0,i0+Nd),d0[i0:i0+Nd],'k',LineWidth=4)
plt.xlabel('sample index',labelpad=15)
plt.xlim(4050,4200)
plt.grid()
plt.tight_layout()
filename='OUTPUT/ideal_waveform.png'
plt.savefig(filename,dpi=200)
plt.show()

# Compute b.
X=np.zeros([Nd,n])
for q in range(n): X[:,q]=np.dot(Cinv,d[(i0-q):(i0-q+Nd)])
b=np.dot(d0[i0:(i0+Nd)],X)

# Plot b.
plt.figure(figsize=(30,10))
plt.plot(b,'k')
plt.plot(b,'ro')
plt.grid()
plt.title('cross-correlation vector b',pad=30)
plt.show()

In [ ]:
# Compute the matrix A.
A=matrix(d,plot=True)

# Solve linear system.
p=np.dot(np.linalg.inv(A),b)

plt.figure(figsize=(30,10))
plt.plot(p,'k')
plt.plot(p,'ro')
plt.grid()
plt.title('Wiener filter coefficients',pad=30)
plt.show()

## 4.2. Application of the filter

It remains to apply the convolutional Wiener filter and to compare with the original data.

In [ ]:
# Compute the prediction.
d_pred=np.zeros(Nd)

for j in range(Nd):
    for i in range(n):
        d_pred[j]+=d[i0+j-i]*p[i]

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(30,10))
plt.plot(np.arange(i0-50,i0+Nd+50),d[i0-50:i0+Nd+50],'--k')
plt.plot(np.arange(i0,i0+Nd),d[i0:i0+Nd],'k',LineWidth=4)
plt.plot(np.arange(i0,i0+Nd),d_pred,'r',LineWidth=3)
plt.grid()
plt.xlim([i0-50,i0+Nd+50])
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index')
plt.show()

## 4.3. Filtering a different time series

Now that we have seen how this works, we look at a different time series from our dataset.

In [ ]:
# Index of the trace.
i_trace_new=363
# Pick trace.
d_new=cct_filt_down[i_trace_new,:]

# Plot individual trace.
plt.figure(figsize=(30,10))
plt.plot(d_new,'k')
plt.grid()
plt.xlabel('sample index')
plt.show()

We need to solve for the new set of filter coefficients. Quite often, the filter coefficients for different time series look pretty similar, and so one may actually also reuse them.

In [ ]:
# Solve for the Wiener filter coefficients.
# Compute the matrix A.
A=matrix(d_new,plot=True)

# Solve linear system.
p=np.dot(np.linalg.inv(A),b)

# Compute the prediction.
d_pred=np.zeros(Nd)

for j in range(Nd):
    for i in range(n):
        d_pred[j]+=d_new[i0+j-i]*p[i]

Again we compare the original and the filtered time series.

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(30,10))
plt.plot(np.arange(i0-50,i0+Nd+50),d_new[i0-50:i0+Nd+50],'--k')
plt.plot(np.arange(i0,i0+Nd),d_new[i0:i0+Nd],'k',LineWidth=4)
plt.plot(np.arange(i0,i0+Nd),d_pred,'r',LineWidth=3)
plt.grid()
plt.xlim([i0-50,i0+Nd+50])
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index')
plt.show()

## 5. Repeating this for a part of the record section

Having gained some confidence, we now do this for a bigger part of the record section.

### 5.1. Plot the original data

In [ ]:
# Minimun and maximum trace indices.
ix_min=0
ix_max=1400

# Trace normalisation for plotting.
normalisation=200.0

plt.figure(figsize=(30,20))

for i in np.arange(ix_min,ix_max):
        
    data = cct_filt_down[i,i0:i0+Nd]/normalisation     
    dist_var = (i-1)*dx
    
    plt.plot(t[i0:i0+Nd],(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t[i0:i0+Nd],(scale*data)+dist_var,y2=np.ones(np.shape(t[i0:i0+Nd]))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]',labelpad=20)
plt.ylabel('distance [m]',labelpad=20)
plt.title('raw data',pad=30)
plt.xlim(40.0,43.0)
plt.ylim(1000.0,5000.0)
plt.grid()
plt.tight_layout()
filename='OUTPUT/raw_data.png'
plt.savefig(filename,dpi=200)
plt.show()

### 5.2. Compute Wiener filter coefficients

For each of the time series we compute the Wiener filter coefficients and apply the filter.

In [ ]:
# Plot matrix and coefficients.
plot=False

# Compute the prediction.
d_filt=np.zeros([ix_max-ix_min,Nd])

# Loop through the traces.
for k in range(ix_max-ix_min):
    
    # Pick out the current trace.
    d_new=cct_filt_down[ix_min+k,:]
    
    # Solve for the Wiener filter coefficients.
    A=matrix(d_new,plot)
    p=np.dot(np.linalg.inv(A),b)
    
    if plot:
        plt.figure(figsize=(30,10))
        plt.plot(p,'k')
        plt.plot(p,'ro')
        plt.grid()
        plt.title('Wiener filter coefficients',pad=30)
        plt.show()
    
    # Apply the filter.
    for j in range(Nd):
        for i in range(n):
            d_filt[k,j]+=d_new[i0+j-i]*p[i]

### 5.3. Plot the filtered record section

In [ ]:
plt.figure(figsize=(30,30))

normalisation=25

for i in range(ix_max-ix_min):
        
    data = d_filt[i,:]/normalisation  
    dist_var = (i+ix_min-1)*dx
    
    plt.plot(t[i0:i0+Nd],(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t[i0:i0+Nd],(scale*data)+dist_var,y2=np.ones(np.shape(t[i0:i0+Nd]))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]')
plt.ylabel('distance [m]')
plt.title('filtered data')
plt.grid()
plt.show()

# 6. Application in orthogonal direction for a single time sample index

Apply the filter simply in the other direction, i.e., with distance.

In [ ]:
# Number of Wiener filter coefficients.
n=5
# Number of traces that are included.
Nt=ix_max-ix_min

### 6.1. Pick out the traces for the selected time sample

In [ ]:
# Index of the time sample.
i_sample=245
# Pick time sample, padded by zeroes.
d=np.zeros(Nt+2*n)
d[n:n+Nt]=d_filt[:,i_sample]

# Plot individual trace.
plt.figure(figsize=(30,10))
plt.plot(d,'k')
plt.grid()
plt.xlabel('trace index')
#plt.xlim([350,400])
plt.show()

### 6.2. Estimate noise covariance matrix

We try to estimate the noise covariance matrix from a few traces before the event.

In [ ]:
# Correlation vector.
corr=np.zeros(Nt)

# Minimum and maximum indices of the samples used to estimate the noise covariance.
# This needs to make sure than only those time samples are included that are actual noise.
imin=0
imax=20

# Number of different reference indices with respect to which the correlation vector is computed.
Nref=50

# Loop over time samples over which we average.
for j in np.arange(imin,imax-1):
    # Loop over correlation lags.
    for i in range(Nt):
        # To obtain a more reasonable result, also loop over reference indices.
        for k in range(Nref):
            corr[i]+=np.roll(d_filt[:,j],k)[0]*np.roll(d_filt[:,j],k)[i]

corr/=np.float(Nref*(imax-imin))

# Smooth a bit.
for s in range(2): corr[1:Nt-1]=(2.0*corr[1:Nt-1]+corr[0:Nt-2]+corr[2:Nt])/4.0

In [ ]:
plt.figure(figsize=(30,10))
plt.plot(corr,'k')
plt.grid()
plt.title('average noise cross-correlation',pad=30)
plt.xlabel('trace index')
plt.xlim([-1,Nt])
plt.show()

In [ ]:
C=np.zeros([Nt,Nt])
for i in range(Nt): C[i,:]=np.roll(corr,i)
Cinv=np.linalg.inv(C)

### 6.3. Compute cross-correlation vector

For this, we again pick out a part of the trace that we think is dominated by actual data.

In [ ]:
# Cut out a part of the trace.
d0=d.copy()
d0[:362]=0.0
d0[380:]=0.0

# Smooth a bit.
for s in range(2): d0[1:Nt+2*n-1]=(2.0*d0[1:Nt+2*n-1]+d0[0:Nt+2*n-2]+d0[2:Nt+2*n])/4.0

# Plot the result.
plt.figure(figsize=(30,10))
plt.plot(np.arange(0,0+Nt),d0[0:0+Nt],'k',LineWidth=4)
plt.xlim(350,450)
plt.show()

# Compute b.
X=np.zeros([Nt,n])
for q in range(n): X[:,q]=np.dot(Cinv,d[(n-q):(n-q+Nt)])
b=np.dot(d0[n:(n+Nt)],X)

# Plot b.
plt.figure(figsize=(30,10))
plt.plot(b,'k')
plt.plot(b,'ro')
plt.grid()
plt.title('cross-correlation vector b',pad=30)
plt.show()

## 6.4. Compute auto-correlation matrix

For convenience, we again define a function that computes the matrix.

In [ ]:
def matrix2(d,plot=False):

    # Build the auxiliary matrix X.
    X=np.zeros([Nt,n])
    for q in range(n): X[:,q]=np.dot(Cinv,d[(n-q):(n-q+Nt)])

    # Build matrix A.
    A=np.zeros([n,n])
    for k in range(n): A[:,k]=np.dot(d[(n-k):(n-k+Nt)],X)

    # Plot matrix.
    if plot:
        plt.figure(figsize=(15,15))
        plt.pcolor(A)
        plt.title('auto-correlation matrix A',pad=30)
        plt.show()

    # Return.
    return A

With this, we can then compute and plot $\mathbf{A}$.

In [ ]:
# Compute the matrix A.
A=matrix2(d,plot=True)

# Solve linear system.
p=np.dot(np.linalg.inv(A),b)

plt.figure(figsize=(30,10))
plt.plot(p,'k')
plt.plot(p,'ro')
plt.grid()
plt.title('Wiener filter coefficients',pad=30)
plt.show()

### 6.5. Apply the filter and plot results

In [ ]:
# Compute the prediction.
d_pred=np.zeros(Nt)

for j in range(Nt):
    for i in range(n):
        d_pred[j]+=d[n+j-i]*p[i]

In [ ]:
# Plot the comparison of observed and predicted data.
plt.figure(figsize=(30,10))
plt.plot(np.arange(0,Nt),d[n:n+Nt],'k',LineWidth=4)
plt.plot(np.arange(0,Nt),d_pred,'r',LineWidth=3)
plt.grid()
plt.title('observation/prediction comparison',pad=30)
plt.xlabel('sample index')
plt.show()

# 7. Application in orthogonal direction for all time indices

On the basis of the previous developments, we now apply the filter to all time samples. For simplicity, we keep $\mathbf{b}$ constant.

In [ ]:
# Compute the prediction.
d_filt2=np.zeros(np.shape(d_filt))

# Loop over time sample indices.
for i_sample in range(Nd):
    
    # Pick time sample, padded by zeroes.
    d=np.zeros(Nt+2*n)
    d[n:n+Nt]=d_filt[:,i_sample]

    # Compute the matrix A.
    A=matrix2(d)

    # Solve linear system.
    p=np.dot(np.linalg.inv(A),b)
    
    for j in range(Nt):
        for i in range(n):
            d_filt2[j,i_sample]+=d[n+j-i]*p[i]

In [ ]:
plt.figure(figsize=(30,20))

normalisation=0.7

for i in range(ix_max-ix_min):
        
    data = d_filt2[i,:]/normalisation       
    dist_var = (i+ix_min-1)*dx
    
    plt.plot(t[i0:i0+Nd],(scale*data)+dist_var,'k-', alpha = 0.4)
    plt.fill_between(t[i0:i0+Nd],(scale*data)+dist_var,y2=np.ones(np.shape(t[i0:i0+Nd]))*dist_var,where=(data+dist_var>=dist_var), interpolate=True,fc='k',alpha=0.8)

plt.xlabel('time [s]',labelpad=20)
plt.ylabel('distance [m]',labelpad=20)
plt.xlim(40.0,43.0)
plt.ylim(1000.0,5000.0)
plt.grid()
plt.tight_layout()
filename='OUTPUT/filtered_data.png'
plt.savefig(filename,dpi=200)
plt.show()